In [ ]:
from keras import models, layers
from ngdlm import models as ngldmodels
from ngdlm import utils as ngldutils
from keras.datasets import mnist
import numpy as np

# Train- and test-data.

In [ ]:
(x_input_train, _), (x_input_test, y_outputs_test) = mnist.load_data()
x_input_train = x_input_train.astype("float32") / 255.0
x_input_test = x_input_test.astype("float32") / 255.0
print(x_input_train.shape)
print(x_input_test.shape)

# Plain autoencoder.

In [ ]:
latent_dim = 32

# Create the encoder.
encoder_input = layers.Input(shape=(28, 28))
encoder_output = layers.Reshape((28 * 28,))(encoder_input)

# Create the decoder.
decoder_input = layers.Input(shape=(latent_dim,))
decoder_output = layers.Dense(28 * 28, activation="sigmoid")(decoder_input)
decoder_output = layers.Reshape((28, 28))(decoder_output)

# Create the autoencoder.
ae = ngldmodels.AE(encoder_input, encoder_output, decoder_input, decoder_output, latent_dim=latent_dim)
ae.compile(optimizer='adadelta', loss='binary_crossentropy')
ae.summary()

# Train.
print("Train...")
history = ae.fit(
        x_input_train, x_input_train,
        epochs=512,
        batch_size=128,
        shuffle=True,
        validation_data=(x_input_test, x_input_test)
    )
    
# Evaluate.
print("Evaluate...")
loss = ae.model.evaluate(x_input_test, x_input_test)
print("Loss:", loss)

# Visualizing plain autoencoder.

In [ ]:
# Render history.
print("Rendering history...")
ngldutils.render_history(history)

# Render conconstructions.
print("Rendering reconstructions...")
ngldutils.render_image_reconstructions(ae, x_input_train, 10)

# Render latent-space.
print("Rendering latent-space...")
ngldutils.render_image_latent_space(ae.decoder, 10)

# Variational autoencoder.

In [ ]:
latent_dim = 2

# Create the encoder.
encoder_input = layers.Input(shape=(28, 28))
encoder_output = layers.Reshape((28 * 28,))(encoder_input)
encoder_output = layers.Dense(512, activation="relu")(encoder_output)

# Create the decoder.
decoder_input = layers.Input(shape=(latent_dim,))
decoder_output = layers.Dense(512, activation="relu")(decoder_input)
decoder_output = layers.Dense(28 * 28, activation="sigmoid")(decoder_output)
decoder_output = layers.Reshape((28, 28))(decoder_output)

# Create the autoencoder.
vae = ngldmodels.VAE(encoder_input, encoder_output, decoder_input, decoder_output, latent_dim=latent_dim)
vae.compile(optimizer='adadelta', reconstruction_loss="binary_crossentropy")
vae.summary()

# Train.
print("Train...")
history = vae.fit(
        x_input_train, x_input_train,
        epochs=50,
        batch_size=32,
        shuffle=True,
        validation_data=(x_input_test, x_input_test)
    )
    
# Evaluate.
print("Evaluate...")
loss = vae.model.evaluate(x_input_test, x_input_test)
print("Loss:", loss)

# Visualizing variational autoencoder.

In [ ]:
print("Rendering history...")
ngldutils.render_history(history)

print("Rendering reconstructions...")
ngldutils.render_image_reconstructions(vae, x_input_train, 10)

print("Rendering latent-space...")
ngldutils.render_image_latent_space(vae.decoder, 10)

print("Rendering encodings...")
ngldutils.render_encodings(vae.encoder, 10)